In [ ]:
# Data process
from sklearn import feature_selection
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
# Set random seed
random.seed(2024)
np.random.seed(2024)
## loading Data
data_file_path =
local_data_file_path = 
dataset=pd.read_csv(data_file_path, encoding='utf-8')
dataset_local= pd.read_csv(local_data_file_path, encoding='utf-8')
dataset['data_source'] = 'MIMIC'
dataset_local['data_source'] = 'Local'
dataset_local.head()
features = []
dataset_1 = dataset[features]
dataset_2 = dataset_local[features]

data_all = pd.concat([dataset_1, dataset_2], axis=0)
data_all
## categorical variable
data_all.columns = [name.replace('.',' ') for name in data_all.columns]

# drop_first=True indicates that the first category is discarded to avoid information redundancy
data_all = pd.get_dummies(data_all,drop_first=True)

data_all.columns
data_all
## split,train， test， test_external
from sklearn.model_selection import train_test_split

data_internal = pd.DataFrame(data_all[data_all["data_source_MIMIC"] == 1])
data_external = pd.DataFrame(data_all[data_all["data_source_MIMIC"] == 0])

index_train, index_test = train_test_split(range(data_internal.shape[0]), test_size = 0.3)

# split dataset
data_train = data_internal.loc[index_train].reset_index( drop = True )
data_test_internal  = data_internal.loc[index_test].reset_index( drop = True )

data_train.drop(columns=['data_source_MIMIC'],inplace=True)
data_test_internal.drop(columns=['data_source_MIMIC'],inplace=True)
data_external.drop(columns=['data_source_MIMIC'],inplace=True)
## Save Data
data_train.to_csv('D:/JupyterLab/AKI/Data/data_preprocess/data_train.csv')
data_test_internal.to_csv('D:/JupyterLab/AKI/Data/data_preprocess/data_test.csv')
data_external.to_csv('D:/JupyterLab/AKI/Data/data_external.csv', encoding='utf-8')
## Load Data
data_train = pd.read_csv('D:/JupyterLab/AKI/Data/data_preprocess/data_train.csv', encoding='utf-8')
data_test = pd.read_csv('D:/JupyterLab/AKI/Data/data_preprocess/data_test.csv', encoding='utf-8')
data_external = pd.read_csv('D:/JupyterLab/AKI/Data/data_external.csv', encoding='utf-8')
from collections import Counter
# def column
column_name = ['Age', 'Weight', 'SBP', 'DBP',
       'Spo2', 'Glucose', 'SIRS', 'SAPSII', 'SOFA', 'Urine_output', 'RBC',
       'WBC', 'Hemoglobin', 'ACEI', 'Beta_blockers', 'ARB', 'Potassium',
       'Sodium', 'SCR', 'LDL_C', 'Loop_diuretics', 'Congestive_heart_failure',
       'Diabetes', 'Malignancy', 'BUN', 'Hypertension', 'AIP', 'Gender_M']
event_column = 'aki_stage'

X_train, y_train = data_train[column_name], data_train[event_column]
X_test, y_test = data_test[column_name], data_test[event_column]
X_test_external, y_test_external = data_external[column_name], data_external[event_column]
## Lasso
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score
# Define the scope of C
C_values = np.logspace(-4, 0, 20)
cv_scores_mean = []
cv_scores_std = []
## Find the optimal C and the C for the 1-SE rule
# Calculate cross-validation scores (5-fold, repeated 3 times)
for C in C_values:
    model = LogisticRegression(
        penalty='l1',
        solver='saga', 
        C=C,
        max_iter=1000,
        random_state=500
    )
    scores = cross_val_score(
        model, X_train, y_train, 
        cv=10, 
        scoring='roc_auc' 
    )
    cv_scores_mean.append(np.mean(scores))
    cv_scores_std.append(np.std(scores))
# Find the optimal C and the C for the 1-SE rule
best_C_index = np.argmax(cv_scores_mean)
best_C = C_values[best_C_index]
one_se_C = C_values[
    np.where(cv_scores_mean >= cv_scores_mean[best_C_index] - cv_scores_std[best_C_index])[0][0]
]
# Print result
print(f"Best C (λ_min): {best_C:.4f}")
print(f"1-SE rule C (λ_1se): {one_se_C:.4f}")
## Feature selection results
# --- Feature selection results ---
def get_selected_features(C_value):
    model = LogisticRegression(
        penalty='l1', solver='saga', 
        C=C_value, max_iter=10000, 
        random_state=200
    ).fit(X_train, y_train)
    selected = X_train.columns[model.coef_[0] != 0].tolist()
    return selected
# Output the features selected by the two C values
print("\n=== Feature selection results ===")
print(f"Number of features (original): {X_train.shape[1]}")
print(f"Number of features (C_min): {len(get_selected_features(best_C))}")
print(f"Number of features (C_1se): {len(get_selected_features(one_se_C))}")

print("\nSelected features (C_min):", get_selected_features(best_C))
print("\nSelected features (C_1se):", get_selected_features(one_se_C))
lasso_features = get_selected_features(best_C)
## Boruta
import numpy as np
import pandas as pd
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns
seed = 42 
np.random.seed(seed)
#Initialize model
rf = RandomForestClassifier(
    n_jobs=-1,
    class_weight='balanced',
    max_depth=5,
    random_state=seed  
)
#Boruta parameter settings
boruta_params = {
    'estimator': rf,
    'n_estimators': 'auto',
    'verbose': 2,
    'random_state': seed,  
    'max_iter': 30,      
    'perc': 100          
}
def run_boruta(X, y, n_runs=20):
    ranking_df = pd.DataFrame(columns=X.columns)
    for i in range(n_runs):
        print(f"\n=== Run {i+1}/{n_runs} ===")
        boruta_params['random_state'] = seed + i  
        selector = BorutaPy(**boruta_params)
        selector.fit(X.values, y.values.ravel()) 
        ranking_df.loc[i] = selector.ranking_
    
    return ranking_df
ranking_df = run_boruta(X_train, y_train, n_runs=20)
### Perform feature selection
ranking_df.to_csv('boruta_rankings.csv')  # Save the ranking results

# Analysis Results
# Calculate median rank
median_ranks = ranking_df.median().sort_values()

# Define selection criteria (indicating selection if median ≤ 2)
selected_features = median_ranks[median_ranks <= 2].index.tolist()
print("\nUltimately selected features:", selected_features)
boruta_features=selected_features 
## visualization
## Lasso coefficient path plot
from sklearn.linear_model import lasso_path
eps = 1e-6 
alphas, coefs, _ = lasso_path(
    X_train.values, y_train.values,
    eps=eps,        
    n_alphas=100,   
    max_iter=10000
)
# Draw path map
plt.figure(figsize=(10, 6))
for i in range(coefs.shape[0]):
    plt.plot(np.log10(alphas), coefs[i, :], 
             label=X_train.columns[i], linewidth=1.5)
plt.axvline(np.log10(1 / best_C), color='black', linestyle='--', label=r'$C_{min}$')
plt.axvline(np.log10(1 / one_se_C), color='blue', linestyle='--', label=r'$C_{1se}$')
x_min = min(np.log10(alphas).min(), np.log10(1 / best_C), np.log10(1 / one_se_C))
x_max = np.log10(alphas).max()
plt.xlim([x_min, x_max])
plt.xlabel('log10(lambda)') 
plt.ylabel('Coefficient value')
plt.title('Lasso Coefficient Paths')
plt.legend(loc='lower center',  bbox_to_anchor=(0.50, -0.40), ncol=len(X_train.columns)//2, fancybox=True, shadow=True)
plt.grid(True)
plt.subplots_adjust(left=0.1, right=0.9, bottom=0.3, top=0.9)  
plt.savefig('D:/JupyterLab/AKI/Result/Fig/lasso_Coeffcient_Paths2.0.pdf', dpi=800, bbox_inches='tight')
plt.show()
## Lasso cross-validation curve
plt.figure(figsize=(15, 6)) 
colors = plt.cm.rainbow(np.linspace(0, 1, len(C_values)))

for i, (mean_score, std_score, C) in enumerate(zip(cv_scores_mean, cv_scores_std, C_values)):
    if i < len(X_train.columns):
        feature_name = X_train.columns[i]
    else:
        break
    plt.errorbar(np.log10(1 / C), mean_score, yerr=std_score,
                 fmt='o', color=colors[i], ecolor='lightgray', capsize=6,
                 label=feature_name)
plt.xlabel('log10(lambda)') 
plt.ylabel('Partial Likelihood Deviance')  
plt.title('Lasso Logistic Regression: CV Score vs. log10(lambda)')
plt.legend(loc='lower center', bbox_to_anchor=(0.40, -0.32), ncol=max(len(C_values) // 5, 1), fancybox=True, shadow=True)
plt.grid(True, linestyle='--', alpha=0.5)
plt.savefig('D:/JupyterLab/AKI/Result/Fig/lasso_Selection_no2.0.pdf', dpi=800, bbox_inches='tight')
plt.show()
### Boruta boxplot
plt.figure(figsize=(15, 8))
sns.boxplot(data=ranking_df[median_ranks.index], palette="Blues")
plt.axhline(y=2.5, color='r', linestyle='--', label='Selection Threshold')
plt.xticks(rotation=90)
plt.title(f"Boruta Feature Selection")
plt.legend()
plt.tight_layout()

plt.savefig('D:/JupyterLab/AKI/Result/Fig/boruta_results2.0.pdf', dpi=800, bbox_inches='tight')
plt.show()
### Intersection network diagram
import networkx as nx
lasso_features = get_selected_features(best_C)

if lasso_features is None:
    print(("lasso_features is None. Please check the feature selection process."). ")
else:
    # Create a set for finding the intersection
    boruta_set = set(boruta_features)  # Boruta feature set
    lasso_set = set(lasso_features)    # Lasso feature set
    intersection = boruta_set.intersection(lasso_set)  # intersection of two sets
    boruta_only = boruta_set - intersection 
    lasso_only = lasso_set - intersection    # Features selected only by Lasso

    print("Features selected by both methods:", intersection)
    print("Only features selected by Boruta:", boruta_only)
    print("Only features selected by Lasso:", lasso_only)
G = nx.Graph()
for feature in boruta_only:
    G.add_edge('Boruta', feature, color='lightcoral')  
    G.add_edge('Lasso', feature, color='lightblue')  
for feature in intersection:
    G.add_edge('Boruta', feature, color='lightcoral')  
    G.add_edge('Lasso', feature, color='lightblue')
edge_colors = [data['color'] for _, _, data in G.edges(data=True)]
node_colors = []
for node in G.nodes():
    if node == 'Boruta':
        node_colors.append('lightcoral') 
    elif node == 'Lasso':
        node_colors.append('lightblue')  
    elif node in boruta_only:
        node_colors.append('lightcoral')  
    elif node in lasso_only:
        node_colors.append('lightblue')  
        node_colors.append('plum')     
plt.figure(figsize=(10, 10))
pos = nx.spring_layout(G, seed=42) 
nx.draw_networkx(
    G,
    pos,
    edge_color=edge_colors,
    node_color=node_colors,
    with_labels=True,
    node_size=1000,
    font_size=10,
    edgecolors='none' 
)
plt.title('Feature Selection by Boruta and Lasso')

plt.savefig('D:/JupyterLab/AKI/Result/Fig/feature_selection_boruta_lasso5.pdf', format='pdf', dpi=300)
pd.Series(list(intersection)).to_csv('D:\\JupyterLab\\AKI\\Result\\Fig\\selected_features_intersection.csv', index=False)


In [ ]:
# Data process
from sklearn import feature_selection
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
## Load Data
data_train = pd.read_csv('D:/JupyterLab/AKI/Data/data_preprocess/data_train.csv', encoding='utf-8')
data_test = pd.read_csv('D:/JupyterLab/AKI/Data/data_preprocess/data_test.csv', encoding='utf-8')
data_external = pd.read_csv('D:/JupyterLab/AKI/Data/data_external.csv', encoding='utf-8')
data_train.columns
### 用筛选的变量
from collections import Counter
# def column
column_name = ['Age', 'Weight', 'Glucose', 'SBP', 'DBP',
               'SpO2', 'SCR', 'Urineoutput', 'SAPSII', 'SIRS', 'SOFA',  'ARNI',
               'Loop diuretics','Congestiveheartfailure', 'WBC', 'BUN',  'AIP', ]
event_column = 'aki_stage'

X_train, y_train = data_train[column_name], data_train[event_column]
X_test, y_test = data_test[column_name], data_test[event_column]
X_test_external, y_test_external = data_external[column_name], data_external[event_column]
# Model Turning
## 1. XGboost
import xgboost as xgb
from sklearn.datasets import make_classification
from sklearn.model_selection import  RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score, classification_report, roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, RandomizedSearchCV
### XGboost Turning (random search)
# 定义参数空间，加入正则化参数
random_param_dist = {
    'learning_rate': np.linspace(0.01, 0.3, 10),
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [3, 4, 5, 6, 7],
    'min_child_weight': [1, 2, 3, 4],
    'subsample': np.linspace(0.5, 1, 5),
    'colsample_bytree': np.linspace(0.5, 1, 5),
    'reg_alpha': np.linspace(0, 1, 5),  
    'reg_lambda': np.linspace(1, 3, 5)  
}
# 初始化XGBoost分类器
xgb_classifier = xgb.XGBClassifier(objective='binary:logistic')
cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
## 调参
random_search = RandomizedSearchCV(
    estimator=xgb_classifier,
    param_distributions=random_param_dist,
    n_iter=50,
    cv=cv,
    scoring='roc_auc',
    n_jobs=4,
    random_state=100
)
random_search.fit(X_train, y_train)
# 打印随机搜索的最佳参数
print(random_search.best_params_)
#Obtain the best model
random_search.fit(X_train, y_train)
best_xgb_model = random_search.best_estimator_
#Print the optimal parameters for random search
Print ("Random search for best parameters:", random_dearch. best_rams_)
print("Best Score : ", random_search.score(X_test, y_test))
### Save XGBoost Model
from joblib import dump, load
model_path = r'D:\JupyterLab\AKI\Model\XGBoost_model4.joblib'
dump(best_xgb_model, model_path)
print(f"The model has been saved to {model_path}")
## 2. SVM Model
from itertools import cycle
import seaborn as sns
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn import svm
from sklearn.svm import SVC, SVR
from sklearn.metrics import cohen_kappa_score, roc_curve, auc
from sklearn.metrics import confusion_matrix, classification_report
from mlxtend.plotting import plot_decision_regions
#### K折交叉验证
#### 线性核函数算法
param_grid_linear = {'C': [0.1, 1, 10]}  
cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
model_linear = GridSearchCV(SVC(kernel="linear", random_state=100), param_grid_linear, cv=cv)
model_linear.fit(X_train, y_train)
print("Best Parameters for Linear Kernel: ", model_linear.best_params_)
best_model_linear = model_linear.best_estimator_
print("Best Score for Linear Kernel: ", best_model_linear.score(X_test, y_test))
from joblib import dump, load
# 保存模型
model_path = r'D:\JupyterLab\AKI\Model\SVM_model_linear4.joblib'
dump(best_model_linear, model_path)

#### 多项式核函数算法
param_grid_poly = {'C': [0.1, 1, 10], 'gamma': [0.01, 0.1, 1]}
cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
model_poly = GridSearchCV(SVC(kernel="poly", random_state=123), param_grid_poly, cv=cv)
model_poly.fit(X_train, y_train)
print("Best Parameters for Polynomial Kernel: ", model_poly.best_params_)
best_model_poly = model_poly.best_estimator_
print("Best Score for Polynomial Kernel: ", best_model_poly.score(X_test, y_test))
print("Number of Support Vectors for Polynomial Kernel: ", len(best_model_poly.support_))
from joblib import dump, load
# 保存模型
model_path = r'D:\JupyterLab\AKI\Model\SVM_model_poly4.joblib'
dump(best_model_poly, model_path)

#### 径向基函数算法
param_grid_poly = {'C': [0.1, 1, 10], 'gamma': [0.01, 0.1, 1]}
cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
model_rbf = GridSearchCV(SVC(kernel="rbf", random_state=200), param_grid_poly, cv=cv)  
print("Best Parameters for RBF Kernel: ", model_rbf.best_params_)
best_model_rbf = model_rbf.best_estimator_
print("Best Score for RBF Kernel: ", best_model_rbf.score(X_test, y_test))
print("Number of Support Vectors for RBF Kernel: ", len(best_model_rbf.support_))
from joblib import dump, load

model_path = r'D:\JupyterLab\AKI\Model\SVM_model_rbf4.joblib'
dump(best_model_rbf, model_path)

#### sigmoid核函数算法
param_grid_poly = {'C': [0.1, 1, 10], 'gamma': [0.01, 0.1, 1]}
cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
model_sigmoid = GridSearchCV(SVC(kernel="sigmoid", random_state=100), param_grid_poly, cv=cv)  
model_sigmoid.fit(X_train, y_train)
print("Best Parameters for Sigmoid Kernel: ", model_sigmoid.best_params_)
best_model_sigmoid = model_sigmoid.best_estimator_
print("Best Score for Sigmoid Kernel: ", best_model_sigmoid.score(X_test, y_test))
print("Number of Support Vectors for Sigmoid Kernel: ", len(best_model_sigmoid.support_))
from joblib import dump, load
# 保存模型
model_path = r'D:\JupyterLab\AKI\Model\SVM_model_sigmoid4.joblib'
dump(best_model_sigmoid, model_path)

## 3. Logistic
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import RandomizedSearchCV, KFold
from scipy.stats import loguniform  
param_dist = {
    'C': loguniform(1e-4, 100), 
    'penalty': ['l1', 'l2'],    
    'solver': ['liblinear', 'saga'],
    'max_iter': [100, 500, 1000], 
    'class_weight': [None, 'balanced']  
}
cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
### 随机搜索
model = LogisticRegression()


random_search = RandomizedSearchCV(
    model,
    param_distributions=param_dist,
    n_iter=50, 
    cv=cv ,       
    scoring='accuracy',
    random_state=200,
    n_jobs=4    
)
random_search.fit(X_train, y_train)
### 训练逻辑回归模型

random_search.fit(X_train, y_train)


print("Best CV Score:", random_search.best_score_) 
print("Best Params:", random_search.best_params_) 


model_logistic = random_search.best_estimator_
test_score = model_logistic.score(X_test, y_test)
print("True Test Score:", test_score)  
### Save Logistict model
from joblib import dump, load
# 保存模型
model_path = r'D:\JupyterLab\AKI\Model\Logistic_model4.joblib'
dump(model_logistic, model_path)

## 4. Elastic Net Regression
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import make_classification
from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
### 随机搜索调参
# 创建分层K折交叉验证对象（保持类别分布）
cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=100)
# 创建弹性网络逻辑回归模型
logistic_elastic_net = LogisticRegression(
    penalty='elasticnet',
    solver='saga',       
    random_state=42,
    max_iter=1000
)  
# 定义参数网格
param_grid = {
    'solver': ['saga'],
    'penalty': ['elasticnet'],
    'l1_ratio': uniform(0, 1), 
    'C': uniform(0.01, 10),     
    'class_weight': [None, 'balanced']  
}
# 创建随机搜索对象
random_search = RandomizedSearchCV(
    estimator=logistic_elastic_net,
    param_distributions=param_grid,
    n_iter=50,            
    cv=cv,  # 使用10折
    scoring='roc_auc',    
    n_jobs=4,           
    random_state=100,
    verbose=2,
    refit=True           
)
random_search.fit(X_train, y_train)
# 输出结果
print("最佳参数:", random_search.best_params_)
print("交叉验证AUC:", random_search.best_score_)
print("测试集AUC:", random_search.score(X_test, y_test))
# 使用随机搜索最佳参数创建模型
best_en_model = random_search.best_estimator_
### Save Elastic Net Regression model
from joblib import dump, load
model_path = r'D:\JupyterLab\AKI\Model\Elastic_Net_Regression_model4.joblib'
dump(best_en_model, model_path)

## 5.AdaBoost
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.inspection import PartialDependenceDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from mlxtend.plotting import plot_decision_regions
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import plot_roc_curve 
# 初始化AdaBoost分类器
model_AdaBoostClassifier = AdaBoostClassifier(random_state=100)
# 训练模型
model_AdaBoostClassifier.fit(X_train, y_train)
# 在测试集上评估模型
score = model_AdaBoostClassifier.score(X_test, y_test)
print("AdaBoost算法得分：", score)
### 随机搜索调参
#定义参数
param_dist = {
    'n_estimators': [50, 100, 200, 300], 
    'learning_rate': [0.01, 0.05, 0.1, 0.5, 1.0], 
    'algorithm': ['SAMME', 'SAMME.R'] }
# 设定交叉验证方法
cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=100)
# 初始化随机搜索模型
random_search = RandomizedSearchCV(
    AdaBoostClassifier(random_state=123),
    param_distributions=param_dist,
    n_iter=10,
    cv=cv,
    random_state=42
)
# 在训练集上做K折调参（不碰测试集）
random_search.fit(X_train, y_train)

print("Best CV Score:", random_search.best_score_) 
print("Best Params:", random_search.best_params_)  

best_adb_model = random_search.best_estimator_
test_score = best_adb_model.score(X_test, y_test)
print("True Test Score:", test_score)  
### Save Adaboost model
from joblib import dump, load

# 保存模型
model_path = r'D:\JupyterLab\AKI\Model\AdaBoost_model4.joblib'
dump(best_adb_model, model_path)

## 6.Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score
# 创建一个随机森林分类器
rf = RandomForestClassifier(random_state=42)


param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=123)
### 随机搜索
# 创建 RandomizedSearchCV 对象
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_grid,
    n_iter=50, 
    cv=cv, 
    scoring='roc_auc', 
    random_state=100,
    n_jobs=4 
)

# 进行随机搜索
random_search.fit(X_train, y_train)
#Obtain the optimal parameters
best_params = random_search.best_params_
#Print optimal parameters
print("Best Parameters: ", random_search.best_params_)
#Print the score of the best estimator on the test set
print("Best Score: ", random_search.score(X_test, y_test))

best_rf = RandomForestClassifier(**best_params, random_state=42)
best_rf.fit(X_train, y_train)
best_rf_model = best_rf
### Save Random Forest model
from joblib import dump, load

# 保存模型
model_path = r'D:\JupyterLab\AKI\Model\RF_model4.joblib'
dump(best_rf_model, model_path)
print(f"模型已保存到 {model_path}")
## 7.Ridge
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from scipy.stats import loguniform
#Classification model optimization
ridge_clf = RidgeClassifier(random_state=42)  

#Define parameter distribution
param_dist = {
'alpha': loguniform(1e-4, 1e4),
'class_weight': [None, 'balanced'],
'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg']
}
#Cross validation strategy
cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=100)
### 随机搜索
# 随机搜索
random_search = RandomizedSearchCV(
    estimator=ridge_clf,
    param_distributions=param_dist,
    n_iter=50,
    cv=cv,
    scoring='roc_auc',
    random_state=100,
    n_jobs=4,
    verbose=1
)
random_search.fit(X_train, y_train)

random_search.fit(X_train, y_train)

print("Best CV Score:", random_search.best_score_)  
print("Best Params:", random_search.best_params_)   

best_clf_model = random_search.best_estimator_
test_score = best_clf_model.score(X_test, y_test)
print("True Test Score:", test_score) 
### Save Ridge moel
from joblib import dump, load

# 保存模型
model_path = r'D:\JupyterLab\AKI\Model\Ridge_model4.joblib'
dump(best_clf_model, model_path)

## 8.KNN
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, roc_auc_score, RocCurveDisplay, 
                             confusion_matrix, classification_report)
from scipy.stats import randint

# 交叉验证策略
cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=100)

param_dist = {
    'n_neighbors': randint(1, 100),               
    'weights': ['uniform', 'distance'],         
    'p': [1, 1.5, 2, 3],                         
    'algorithm': ['auto', 'ball_tree', 'kd_tree'],
    'leaf_size': randint(10, 100)        
}

# 初始化KNN模型
knn = KNeighborsClassifier()
### 随机搜索
#配置随机搜索（集成K折交叉验证）
random_search = RandomizedSearchCV(
    estimator=knn,
    param_distributions=param_dist,
    n_iter=50,                 
    cv=cv,        
    scoring='roc_auc',
    n_jobs=4,                  
    random_state=100,
    verbose=2,
    refit=True                  
)
# 执行搜索
random_search.fit(X_train, y_train)

random_search.fit(X_train, y_train)

print("Best CV Score:", random_search.best_score_)  
print("Best Params:", random_search.best_params_)  

best_knn_model  = random_search.best_estimator_
test_score = best_knn_model .score(X_test, y_test)
print("True Test Score:", test_score) 
### Save KNN model
from joblib import dump, load

# 保存模型
model_path = r'D:\JupyterLab\AKI\Model\KNN_model4.joblib'
dump(best_knn_model, model_path)

## 9.SGD
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from scipy.stats import loguniform, uniform
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss
from copy import deepcopy
#初始化基础模型
base_model = SGDClassifier(
    loss='log',  
    penalty='elasticnet',
    random_state=42,
    max_iter=10000, 
    warm_start=False,  
    learning_rate='optimal' )
#定义参数分布（关键参数搜索空间）
param_dist = {
    'alpha': loguniform(1e-6, 1e-2),
    'l1_ratio': uniform(0, 1),
    'eta0': loguniform(1e-4, 1e-1), 
    'power_t': uniform(0.1, 0.5) 
}

# 交叉验证策略
cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=100)
### 随机搜索

random_search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_dist,
    n_iter=10,
    cv=cv,
    scoring='roc_auc',
    n_jobs=3,  
    random_state=30,
    verbose=2,
    error_score='raise' 
)

random_search.fit(X_train, y_train)

print("Best CV Score:", random_search.best_score_) 
print("Best Params:", random_search.best_params_)   

best_sgd_model  = random_search.best_estimator_
test_score = best_sgd_model .score(X_test, y_test)
print("True Test Score:", test_score) 
### Save SGD model
from joblib import dump, load

# 保存模型
model_path = r'D:\JupyterLab\AKI\Model\SGD_model4.joblib'
dump(best_sgd_model, model_path)

## 10.ANN
from sklearn.datasets import make_classification
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

param_dist = {
    'activation': ['relu', 'tanh'],
    'solver': ['sgd'],
    'alpha': [0.0001, 0.001],
    'learning_rate': ['constant'], 
    'learning_rate_init': [0.001, 0.01, 0.1], 
    'max_iter': [200, 300, 1000],
    'hidden_layer_sizes': [(128,), (64,), (128, 64), (64, 32), (128, 64, 32)],
    'batch_size': [32, 64, 128]
}

ann = MLPClassifier(random_state=42)
# 交叉验证策略
cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=100)
### 随机搜索
# 创建随机搜索对象
random_search = RandomizedSearchCV(
    estimator=ann,
    param_distributions=param_dist,
    n_iter=10,
    cv=cv,
    scoring='roc_auc',
    n_jobs=4,
    verbose=2,
    random_state=60
)

random_search.fit(X_train, y_train)

print("Best CV Score:", random_search.best_score_)  
print("Best Params:", random_search.best_params_)  


best_ann_model  = random_search.best_estimator_
test_score = best_ann_model .score(X_test, y_test)
print("True Test Score:", test_score) 
### Save ANN model
from joblib import dump, load

# 保存模型
model_path = r'D:\JupyterLab\AKI\Model\ANN_model4.joblib'
dump(best_ann_model, model_path)

## 11.Bagging
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
import xgboost as xgb
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import (accuracy_score, roc_auc_score, 
                             confusion_matrix, classification_report)
from scipy.stats import uniform, randint
import joblib
import seaborn as sns

# 交叉验证策略
cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=100)
# --------------------- 基础XGBoost模型 ---------------------
base_xgb = xgb.XGBClassifier(
    objective='binary:logistic',
    booster='gbtree',
    eval_metric='auc',
    random_state=42
)

# --------------------- 超参数搜索空间 ---------------------
param_dist = {
    'base_estimator__max_depth': randint(3, 10),
    'base_estimator__learning_rate': uniform(0.01, 0.3),
    'base_estimator__n_estimators': randint(50, 300),
    'base_estimator__subsample': uniform(0.6, 0.4),
    'base_estimator__colsample_bytree': uniform(0.6, 0.4),
    'base_estimator__gamma': uniform(0, 2),
    'n_estimators': randint(5, 20), 
    'max_samples': uniform(0.6, 0.4)
}
# --------------------- 创建Bagging集成 ---------------------
bagging_xgb = BaggingClassifier(
    base_estimator=base_xgb,
    bootstrap=True,
    random_state=100,
    n_jobs=-1
)
### 随机搜索
# --------------------- 随机搜索优化 ---------------------
random_search= RandomizedSearchCV(
    estimator=bagging_xgb,
    param_distributions=param_dist,
    n_iter=30,  # 搜索迭代次数
    cv=cv,       # 交叉验证折数
    scoring='roc_auc',
    random_state=42,
    verbose=2
)

random_search.fit(X_train, y_train)
print("Best CV Score:", random_search.best_score_) 
print("Best Params:", random_search.best_params_)  


# 用最优模型独立评估测试集
best_Bagging_xgb = random_search.best_estimator_
test_score = best_Bagging_xgb .score(X_test, y_test)
print("True Test Score:", test_score)  
### Save Bagging model
from joblib import dump, load
best_Bagging_model=best_Bagging_xgb
# 保存模型
model_path = r'D:\JupyterLab\AKI\Model\Bagging_model4.joblib'
dump(best_Bagging_model, model_path)

## 12. MLP
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, accuracy_score
from scipy.stats import loguniform, randint
mlp = MLPClassifier(max_iter=500, random_state=42)  

param_dist = {
    'hidden_layer_sizes': [(50,), (30, 20)],  
    'activation': ['relu', 'tanh'],
    'alpha': loguniform(1e-5, 1e-2),  
    'learning_rate_init': [0.001, 0.01]  
}

# 交叉验证策略
cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=100)
### 随机搜索
# 随机搜索调参
random_search = RandomizedSearchCV(
    estimator=mlp,
    param_distributions=param_dist,
    n_iter=50,                     
    cv=cv,
    scoring='roc_auc',             
    n_jobs=4,                     
    random_state=100,
    verbose=2
)

random_search.fit(X_train, y_train)

print("Best CV Score:", random_search.best_score_)  
print("Best Params:", random_search.best_params_)  

best_MLP_model  = random_search.best_estimator_
test_score = best_MLP_model .score(X_test, y_test)
print("True Test Score:", test_score)
### Save MLP model
from joblib import dump, load

# 保存模型
model_path = r'D:\JupyterLab\AKI\Model\MLP_model4.joblib'
dump(best_MLP_model, model_path)

## 13. Catboost
from catboost import CatBoostClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
 random_state = 42
# 定义超参数分布
param_dist_cat = {
    'iterations': [500, 1000, 1500],  
    'learning_rate': [0.01, 0.05, 0.1], 
    'depth': [4, 6, 8],              
    'l2_leaf_reg': [1, 3, 5],        
    'eval_metric': ['F1'],           
    'random_seed': [random_state],
    'verbose': [0]
   
}
# 定义 K 折交叉验证

cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=100)

catboost = CatBoostClassifier()
### 随机搜索
# 配置随机搜索
random_search_cat = RandomizedSearchCV(
    estimator=catboost,
    param_distributions=param_dist_cat,
    n_iter=50,                     
    scoring='roc_auc',              
    cv=cv,                         
    n_jobs=4,                      
    verbose=2,
    random_state=100,
    refit=True   )                   
# 执行随机搜索
random_search_cat.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),      
    early_stopping_rounds=20         
)

print("Best CV Score:", random_search_cat.best_score_) 
print("Best Params:", random_search_cat.best_params_)  


best_catboost_model  = random_search_cat.best_estimator_
test_score = best_catboost_model .score(X_test, y_test)
print("True Test Score:", test_score)  
# 评估测试集
best_cat = random_search_cat.best_estimator_
y_pred_cat = best_cat.predict(X_test)
y_proba_cat = best_cat.predict_proba(X_test)[:, 1]  
print(f"\n测试集性能：")
print(f"准确率：{accuracy_score(y_test, y_pred_cat):.4f}")
print(f"F1-score：{f1_score(y_test, y_pred_cat):.4f}")
print(f"AUC-ROC：{roc_auc_score(y_test, y_proba_cat):.4f}")
### Save Catboost model
from joblib import dump, load

# 保存模型
model_path = r'D:\JupyterLab\AKI\Model\Catboost_model4.joblib'
dump(best_catboost_model, model_path)



In [ ]:
# Load model
import tensorflow as tf
from joblib import dump, load
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Load MLP Model
MLP_model = load(r'D:\JupyterLab\AKI\Model\MLP_model4.joblib')
#
## Load ADB Model
ADB_model = load( r'D:\JupyterLab\AKI\Model\AdaBoost_model4.joblib')
#
## Load ANN Model
ANN_model = load(r'D:\JupyterLab\AKI\Model\ANN_model4.joblib')
#
## Load Bagging Model
Bagging_model = load(r'D:\JupyterLab\AKI\Model\Bagging_model4.joblib')
#
## Load en Model
Elastic_Net_Regression_model= load(r'D:\JupyterLab\AKI\Model\Elastic_Net_Regression_model4.joblib')
#
## Load knn Model
KNN_model= load( r'D:\JupyterLab\AKI\Model\KNN_model4.joblib')
#
## Load Logistic Model
Logistic_model= load(r'D:\JupyterLab\AKI\Model\Logistic_model4.joblib')
#
## Load Ridege Model
Rideg_model= load(r'D:\JupyterLab\AKI\Model\Ridge_model4.joblib')
#
## Load SGD Model
SGD_model= load(r'D:\JupyterLab\AKI\Model\SGD_model4.joblib')
#
## Load SVM Model
SVM_model= load(r'D:\JupyterLab\AKI\Model\SVM_model_rbf4.joblib')
#
## Load xgb Model
XGB_model= load(r'D:\JupyterLab\AKI\Model\XGBoost_model5.joblib')
#
## Load RF Model
RF_model= load(r'D:\JupyterLab\AKI\Model\RF_model4.joblib')
#
## Load CAB Model
Catboost_model= load(r'D:\JupyterLab\AKI\Model\Catboost_model4.joblib')

models = {
    "MLP": MLP_model,
    "AdaBoost": ADB_model,
    "Bagging": Bagging_model,
    "Elastic Net Regression": Elastic_Net_Regression_model,
    "ANN": ANN_model,
    "Random Forest": RF_model,
    "KNN": KNN_model,
    "Catboost": Catboost_model,
    "Logistic": Logistic_model,
    "Ridge": Rideg_model,
    "SGD": SGD_model,
    "SVM": SVM_model,
    "XGBoost": XGB_model
}
import matplotlib.pyplot as plt
from sklearn.utils import check_consistent_length
from sklearn.calibration import calibration_curve
from sklearn.metrics import (roc_curve, auc, brier_score_loss, 
                            accuracy_score, recall_score, 
                            precision_score, f1_score, 
                            confusion_matrix)
### Internal
#Generate prediction results
predictions = {}

for model_key, model_ in models.items():
Print (f "\ n Processing model: {model_key}")
try:
       
if hasattr(model_, "predict_proba"):
predictions[f"{model_key}"] = model_.predict_proba(X_test)[:, 1]
else:
predictions[f"{model_key}"]  = model_.decision_function(X_test)

  
    
except Exception as e:
Print (f "Error processing model {model_key}: {str (e)}")
continue    
### External
#Generate prediction results
predictions_external = {}

for model_key, model_ in models.items():
Print (f "\ n Processing model: {model_key}")
try:
#Obtain predicted values
if hasattr(model_, "predict_proba"):
predictions_external[f"{model_key}"] = model_.predict_proba(X_test_external)[:, 1]
else:
predictions_external[f"{model_key}"]  = model_.decision_function(X_test_external)
            
except Exception as e:
Print (f "Error processing model {model_key}: {str (e)}")
continue
##ROC curve
###External ROC
#Create a graphic object
plt.figure(figsize=(8, 8))


for model_key, model_name in models.items():
   
    prediction = predictions_external[model_key]
    fpr, tpr, thresholds = roc_curve(y_test_external, prediction)
    roc_auc_test = auc(fpr, tpr)
    roc_auc = auc(fpr, tpr)
    
    
    plt.plot(fpr, tpr, lw=1, label=f'{model_key} (area = {roc_auc:.4f})')
    

plt.plot([0, 1], [0, 1], color='navy', lw=1, linestyle='--')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title("ROC Curves Comparison (External Validation Set)")
plt.legend(loc="lower right")

plt.show

plt.figure(figsize=(8, 8))

#Traverse each model in order
for model_key, model_name in models.items():
#Obtain the prediction results of the current model
prediction = predictions[model_key]
fpr, tpr, thresholds = roc_curve(y_test, prediction)
roc_auc_test = auc(fpr, tpr)
roc_auc = auc(fpr, tpr)
    
#Draw the ROC curve of the current model
plt.plot(fpr, tpr, lw=1, label=f'{model_key} (area = {roc_auc:.4f})')
    
#Draw diagonal lines as reference
plt.plot([0, 1], [0, 1], color='navy', lw=1, linestyle='--')

#Set graphic properties
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title("ROC Curves Comparison (Internal Test Set)")
plt.legend(loc="lower right")

plt.savefig('D:/JupyterLab/AKI/Result/Fig/test_set_roc_curves5.27.pdf', 
format='pdf', dpi=800, bbox_inches='tight')
## DCA
# 1. Net benefits brought by computational models
def calculate_net_benefit_model(thresh_group, y_pred_score, y_label):
    net_benefit_model = np.array([])
    for thresh in thresh_group:
        y_pred_label = y_pred_score > thresh
        tn, fp, fn, tp = confusion_matrix(y_label, y_pred_label).ravel()
        n = len(y_label)
        net_benefit = (tp / n) - (fp / n) * (thresh / (1 - thresh))
        net_benefit_model = np.append(net_benefit_model, net_benefit)
    return net_benefit_model
# 2. Calculate the net benefits brought by the treat all strategy
def calculate_net_benefit_all(thresh_group, y_label):
net_benefit_all = np.array([])
tn, fp, fn, tp = confusion_matrix(y_label, y_label).ravel()
total = tp + tn
for thresh in thresh_group:
net_benefit = (tp / total) - (tn / total) * (thresh / (1 - thresh))
net_benefit_all = np.append(net_benefit_all, net_benefit)
return net_benefit_all
#Construct a model with poor classification performance
thresh_group = np.arange(0,1,0.001)
###Internal data
predictions['Bagging']
#Internal data
prediction = predictions[model_key]
fpr, tpr, thresholds = roc_curve(y_test, prediction)
net_benefit_model_XGBoost = calculate_net_benefit_model(thresh_group, predictions['XGBoost'], y_test)
net_benefit_model_ANN = calculate_net_benefit_model(thresh_group, predictions['ANN'], y_test)
net_benefit_model_KNN = calculate_net_benefit_model(thresh_group, predictions['KNN'], y_test)
net_benefit_model_Bagging = calculate_net_benefit_model(thresh_group, predictions['Bagging'], y_test)
net_benefit_model_AdaBoost = calculate_net_benefit_model(thresh_group, predictions['AdaBoost'], y_test)
net_benefit_model_Random_Forest = calculate_net_benefit_model(thresh_group, predictions['Random Forest'], y_test)
net_benefit_model_Logistic = calculate_net_benefit_model(thresh_group, predictions['Logistic'], y_test)
net_benefit_model_Catboost = calculate_net_benefit_model(thresh_group, predictions['Catboost'], y_test)
net_benefit_all = calculate_net_benefit_all(thresh_group, y_test)
###Drawing
#Internal Data DCA Curve
Fig, ax=plt. subplots (figsize=(10, 6)) # Adjust chart size

#Define a more discriminative color scheme
colors = [
'#FF7F50', #Coral Red
'#6A5ACD', #Stone slab blue
'#00CED1', #Dark Blue
'#FFA500', #Orange
'#CD5C5C', #Indian Red
'#20B2AA', #Bright Steel Blue
'#9370DB', #Dark Violet
'#3CB371',  # MediumSeaGreen
]

#Draw model curve
models = [
(net_benefit_model_ANN, 'ANN'),
(net_benefit_model_KNN, 'KNN'),
(net_benefit_model_XGBoost, 'XGBoost'),
(net_benefit_model_AdaBoost, 'AdaBoost'),
(net_benefit_model_Catboost, 'Catboost'),
(net_benefit_model_Random_Forest, 'Random Forest'),
(net_benefit_model_Bagging, 'Bagging'),
(net_benefit_model_Logistic, 'Logistic')
]

#Draw model curves using loops
for i, (model, label) in enumerate(models):
Ax. plot (thresh_group, model, color=colors [i], label=label, lw=1.5) # Increase line width

#Draw reference lines
ax.plot(thresh_group, net_benefit_all, color='#808080', label='Treat all', lw=1.5)  #  dark gray
ax.plot((0, 1), (0, 0), color='#808080', linestyle=':', label='Treat none', lw=1.5)


ax.set_xlim(0, 1)
ax.set_ylim(-0.1, 0.6)
ax.set_xlabel(
    xlabel='Threshold Probability',
    fontdict={'family': 'Times New Roman', 'fontsize': 15}
)
ax.set_ylabel(
    ylabel='Net Benefit',
    fontdict={'family': 'Times New Roman', 'fontsize': 15}
)


ax.spines['right'].set_color((0.8, 0.8, 0.8))
ax.spines['top'].set_color((0.8, 0.8, 0.8))

legend = ax.legend(loc='upper right', frameon=True)
frame = legend.get_frame()
frame.set_facecolor('white')
frame.set_edgecolor('#e0e0e0')

plt.tight_layout()  
plt.savefig('D:/JupyterLab/AKI/Result/Fig/DCA_internal5.27.pdf', 
            format='pdf', dpi=300, bbox_inches='tight')

plt.show()
###External data
#Construct a model with poor classification performance
thresh_group = np.arange(0,1,0.001)
#External data
prediction_external = predictions_external[model_key]
fpr, tpr, thresholds = roc_curve(y_test_external, prediction_external)
net_benefit_model_external_XGBoost= calculate_net_benefit_model(thresh_group, predictions_external['XGBoost'], y_test_external)
net_benefit_model_external_ANN= calculate_net_benefit_model(thresh_group, predictions_external['ANN'], y_test_external)
net_benefit_model_external_KNN= calculate_net_benefit_model(thresh_group, predictions_external['KNN'], y_test_external)
net_benefit_model_external_Bagging= calculate_net_benefit_model(thresh_group, predictions_external['Bagging'], y_test_external)
net_benefit_model_external_AdaBoost= calculate_net_benefit_model(thresh_group, predictions_external['AdaBoost'], y_test_external)
net_benefit_model_external_Random_Forest= calculate_net_benefit_model(thresh_group, predictions_external['Random Forest'], y_test_external)
net_benefit_model_external_Logistic= calculate_net_benefit_model(thresh_group, predictions_external['Logistic'], y_test_external)
net_benefit_model_external_Catboost= calculate_net_benefit_model(thresh_group, predictions_external['Catboost'], y_test_external)
#External data
net_benefit_all_external = calculate_net_benefit_all(thresh_group, y_test_external)
###Drawing
#DCA curve of external data
fig, ax = plt.subplots(figsize=(10, 6)) 

colors = [
    '#FF7F50',  
    '#6A5ACD',  
    '#00CED1',  
    '#FFA500',  
    '#CD5C5C',  
    '#20B2AA',  
    '#9370DB',  
    '#3CB371', 
]


models = [
    (net_benefit_model_external_ANN, 'ANN'),
    (net_benefit_model_external_KNN, 'KNN'),
    (net_benefit_model_external_XGBoost, 'XGBoost'),
    (net_benefit_model_external_AdaBoost, 'AdaBoost'),
    (net_benefit_model_external_Catboost, 'Catboost'),
    (net_benefit_model_external_Random_Forest, 'Random Forest'),
    (net_benefit_model_external_Bagging, 'Bagging'),
    (net_benefit_model_external_Logistic, 'Logistic')
]


for i, (model, label) in enumerate(models):
    ax.plot(thresh_group, model, color=colors[i], label=label, lw=1.5, alpha=0.85) 

ax.plot(thresh_group, net_benefit_all_external, color='#808080', label='Treat all', lw=1.5)  
ax.plot((0, 1), (0, 0), color='#808080', linestyle=':', label='Treat none', lw=1.5)


ax.set_xlim(0, 1)
ax.set_ylim(-0.1, 0.4)  
ax.set_xlabel(
    xlabel='Threshold Probability',
    fontdict={'family': 'Times New Roman', 'fontsize': 15}
)
ax.set_ylabel(
    ylabel='Net Benefit',
    fontdict={'family': 'Times New Roman', 'fontsize': 15}
)


ax.spines['right'].set_color((0.8, 0.8, 0.8))
ax.spines['top'].set_color((0.8, 0.8, 0.8))
ax.spines['left'].set_linewidth(1.2)  
ax.spines['bottom'].set_linewidth(1.2)  


legend = ax.legend(loc='upper right', frameon=True, ncol=2)
frame = legend.get_frame()
frame.set_facecolor('white')
frame.set_edgecolor('#e0e0e0')
frame.set_alpha(0.95)  

ax.grid(True, axis='y', linestyle='--', alpha=0.3)


plt.tight_layout()  
#plt.savefig('D:/JupyterLab/AKI/Result/Fig/DCA_external5.27.pdf', 
#           format='pdf', dpi=800, bbox_inches='tight')
plt.show()

plt.rcParams['font.family'] = 'Arial'

selected_models = {
    "AdaBoost": ADB_model,
    "Bagging": Bagging_model,
    "ANN": ANN_model,
    "Random Forest": RF_model,
    "KNN": KNN_model,
    "CatBoost": Catboost_model,
    "Logistic": Logistic_model,
    "XGBoost": XGB_model
}

predictions_internal_selected = {}
predictions_external_selected = {}

for model_key, model in selected_models.items():
    print(f"Processing model: {model_key}")
    try:
       #Internal data prediction
if hasattr(model, "predict_proba"):
predictions_internal_selected[model_key] = model.predict_proba(X_test)[:, 1]
else:
predictions_internal_selected[model_key] = model.decision_function(X_test)

#External data prediction
        if hasattr(model, "predict_proba"):
            predictions_external_selected[model_key] = model.predict_proba(X_test_external)[:, 1]
        else:
            predictions_external_selected[model_key] = model.decision_function(X_test_external)
    except Exception as e:
        print(f"Error occurred while processing model {model_key}: {e}")
        continue

def safe_normalize(pred):
    pred = np.array(pred)
    if pred.min() == pred.max():  
        return np.clip(pred, 0, 1)
    return (pred - pred.min()) / (pred.max() - pred.min())

print("Internal Data Brier Scores:")
for model_key, score in brier_scores_internal.items():
    print(f"{model_key}: {score:.4f}")

### Brier Score

print("External Data Brier Scores:")
for model_key, score in brier_scores_external.items():
    print(f"{model_key}: {score:.4f}")
## Variable Importance
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
import os
import pandas as pd
from sklearn.inspection import permutation_importance

def variableImportance(model, X, y):
    result = permutation_importance(
        model, X, y, n_repeats=10, random_state=42, n_jobs=4
    )
    return result.importances_mean

imp = pd.DataFrame(
    {
        model_name: variableImportance(model, X_test, y_test)
        for model_name, model in selected_models.items()
    },
    index=column_name
)
imp['Average'] = imp.mean(axis=1)

def plotFeatureImportance(a):
    mpl.rcParams['figure.dpi'] = 300
    a = a.sort_values(by=['Average'], ascending=False)
    fig, ax = plt.subplots(figsize=(8, 10))

    def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
        new_cmap = colors.LinearSegmentedColormap.from_list(
            'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
            cmap(np.linspace(minval, maxval, n)))
        return new_cmap

    im = ax.imshow(a[list(selected_models.keys()) + ['Average']],
                   cmap=truncate_colormap(cm.get_cmap("hot"), minval=0.4, maxval=1), norm=colors.SymLogNorm(1e-3))
    cbar = ax.figure.colorbar(im, ax=ax)
    cbar.ax.set_ylabel("Permutation Feature Importance", rotation=-90, va="bottom")
    cbar.set_ticks([0.1, 0.01, 0.001, 0, -0.001])
    cbar.set_ticklabels(['10', "1", "0.1", 0, '-1'])

    ax.set_yticks(np.arange(len(a)))
    ax.set_xticks(np.arange(len(a.columns)))
    ax.set_xticklabels(list(selected_models.keys()) + ['Average'])
    ax.set_yticklabels(a.index)
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    fig.tight_layout()

plotFeatureImportance(imp)
plt.savefig('D:/JupyterLab/AKI/Result/Fig/feature_importance5.27.pdf', 
           format='pdf', dpi=300, bbox_inches='tight')
## Shap
import shap
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
### XGBOOST
import shap
import matplotlib.pyplot as plt
import numpy as np
import shap
import matplotlib.pyplot as plt
import numpy as np


def XGBoost_prediction(X):
    return XGB_model.predict(X)

#XGBoost_explainer = shap.TreeExplainer(XGBoost_prediction, X_train)
XGBoost_explainer = shap.TreeExplainer(XGB_model)
XGBoost_shap_values = XGBoost_explainer.shap_values(X_train)

XGBoost_shap_values = XGBoost_explainer(X_train)


num_features = X_train.shape[1]


plt.figure(figsize=(10, 0.5 * num_features))
shap.summary_plot(
    XGBoost_shap_values.values,  
    X_train,
    max_display=num_features,
    show=False
)


ax = plt.gca()
ax.set_xlim(-0.6, 0.6)  
ticks = np.arange(-0.6, 0.6 + 0.2, 0.2) 
ax.set_xticks(ticks)
ax.set_xticklabels([f"{tick:.1f}" for tick in ticks], rotation=0, fontsize=8)

ax.grid(False)  
ax.yaxis.grid(False) 

plt.savefig(
    "D:/JupyterLab/AKI/Result/Fig/XGBoost_shap_summary_all_features610.pdf",
    format="pdf",
    dpi=800,
    bbox_inches="tight"
)
plt.close()
plt.show()

shap.dependence_plot(
    "AIP",                           
    XGBoost_shap_values.values,       
    X_train,                         
    interaction_index=None,          
    show=False                        
)


plt.title("SHAP Dependence Plot for AIP", fontsize=12)
plt.xlabel("AIP Value", fontsize=10)
plt.ylabel("SHAP Value (Impact on Prediction)", fontsize=10)


plt.show()
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, roc_auc_score, cohen_kappa_score
import numpy as np
import pandas as pd
#Define the function for calculating indicators
def calculate_metrics(y_true, y_pred_probs):
#Generate category predictions based on predicted probabilities
If np. min (y_pred_debs)<0: # Decision function values may have negative numbers
y_pred = (y_pred_probs >= 0).astype(int)
Else: # Probability values between 0-1
y_pred = (y_pred_probs >= 0.5).astype(int)
    
metrics = {}
    
#Basic indicators
Metrics [Error Rate]=1- Accuracy_score (y_true, y_pred)
Metrics [accuracy]=accuracy_store (y_true, y_pred)
Metrics [precision]=precision_store (y_true, y_pred)
Metrics [recall rate]=recall_stcore (y_true, y_pred)
Metrics ['F1 value ']=f1_score (y_true, y_pred)
    
# AUC
Metrics ['AUC value ']=roc_ auc_score (y_true, y_pred_debs)
    
#Cohen Kappa
Metrics [Cohen score]=Cohen, kappa, score (y_true, y_pred)
    
#Confusion matrix
cm = confusion_matrix(y_true, y_pred)
Metrics [confusion matrix]=f "TN={cm [0,0]}, FP={cm [0,1]}, FN={cm [1,0]}, TP={cm [1,1]}"
    
return metrics

#Initialization result storage
internal_results = {}
external_results = {}

#Handling internal test sets
for model_key, preds in predictions.items():
try:
#Calculate indicators
internal_results[model_key] = calculate_metrics(
y_test, 
preds
)
except Exception as e:
Print (f "Error calculating internal metrics of model {model_key}: {str (e)}")
Internal_desults [model_key]={Error: str (e)}

#Processing external validation sets
for model_key, preds in predictions_external.items():
try:
#Calculate indicators
external_results[model_key] = calculate_metrics(
y_test_external, 
preds
)
except Exception as e:
Print (f "Error calculating external metrics for model {model_key}: {str (e)}")
External_ results [model_key]={Error: str (e)}



In [ ]:
def plot_idi(y_truth: np.ndarray, ref_model: np.ndarray, new_model: np.ndarray,
             thresholds: Dict[float, str],num_bootstraps: int = 1000, show: bool = True, save: bool = False) ->  Dict:
    """
    Plot the Integrated Discrimination Improvement (IDI) curve.

    Args:
        y_truth (np.ndarray): Ground truth labels.
        ref_model (np.ndarray): Reference model predictions.
        new_model (np.ndarray): New model predictions.
        thresholds (dict): Dictionary of thresholds with their corresponding labels.
        num_bootstraps (int, optional): Number of bootstrap iterations. Default is 1000.
        save (bool, optional): Whether to save the plot. Default is False.
    Returns:
        Dict: A dictionary containing the plot, IDI statistics, and NRI values.
            - 'plot': The matplotlib figure object of the IDI curve.
            - 'IDI': A dictionary of IDI statistics including overall IDI, IDI for events, and IDI for nonevents.
            - 'NRI': A dictionary of NRI values at specified thresholds.
    Note:
        - This function requires the `matplotlib`, `numpy`, and `sklearn` packages.
        - The thresholds dictionary should contain values between 0 and 100, representing percentages.
    """
    
    ref_fpr, ref_tpr, ref_thresholds = metrics.roc_curve(y_truth, ref_model)
    new_fpr, new_tpr, new_thresholds = metrics.roc_curve(y_truth, new_model)
    base, mean_tprs, mean_fprs,_,_ = bootstrap_results(y_truth, new_model, num_bootstraps)
    base2, mean_tprs2, mean_fprs2,_,_ = bootstrap_results(y_truth, ref_model, num_bootstraps)
    is_pos, is_neg, idi_event = area_between_curves(mean_tprs, mean_tprs2)
    ip_pos, ip_neg, idi_nonevent = area_between_curves(mean_fprs2, mean_fprs)

    plt.ioff()
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111)
    #ax = ax.axes()
    lw = 2
    ax.plot(base, mean_tprs, 'steelblue', alpha=0.5, label='Events New (New)')
    ax.plot(base, mean_fprs, 'deeppink', alpha=0.5, label='Nonevents New (New)')
    ax.plot(base2, mean_tprs2, 'steelblue', alpha=0.7, linestyle='--', label='Events Reference (Ref)')
    ax.plot(base2, mean_fprs2, 'deeppink', alpha=0.7, linestyle='--', label='Nonevents Reference (Ref)')
    ax.fill_between(base, mean_tprs, mean_tprs2, color='steelblue', alpha=0.2,
                     label='Integrated Sensitivity (area = %0.2f)' % idi_event)
    ax.fill_between(base, mean_fprs, mean_fprs2, color='deeppink', alpha=0.2,
                     label='Integrated Specificity (area = %0.2f)' % idi_nonevent)

    def nri_annotation(plt, threshold, label, color):
        """
        Annotate the plot with NRI information for a given threshold.
        
        This function adds vertical lines, text annotations, and arrow annotations to the plot
        to represent the Net Reclassification Improvement (NRI) for events and nonevents at a specific threshold.
        The function modifies the provided matplotlib plot in place.
        
        Args:
        plt (matplotlib.pyplot): The matplotlib plotting module or a matplotlib Axes object.
        threshold (int or float): The threshold value at which to calculate and annotate the NRI.
                                  This should correspond to a point on the ROC curve.
        label (str): The label to use for the vertical line annotation at the threshold.
        color (str or tuple): The color to use for the annotations. This can be a named color,
                              a hex color code, or an RGB tuple.
        
        Returns:
        tuple: A tuple containing the threshold, NRI for events, NRI for nonevents, and total NRI.
               Formatted as (threshold, NRI_events, NRI_nonevents, total_NRI).
        
        Note:
        - This function is intended to be used as part of the `plot_idi` function.
        - The `threshold` argument is expected to be within the range of the model's prediction scores.
        """
        x_pos = base[threshold]
        x_offset = 0.02
        x_offset2 = x_offset
        text_y_offset = 0.01
        text_y_offset2 = text_y_offset
        if threshold == 2:
            text_y_offset = 0.04
            text_y_offset2 = 0.04
            x_offset2 = 0.05
            
        text_y_events = np.mean([mean_tprs2[threshold], mean_tprs[threshold]]) + text_y_offset
        text_y_nonevents = np.mean([mean_fprs[threshold], mean_fprs2[threshold]]) + text_y_offset2

        ax.axvline(x=threshold/100, color=color, linestyle='--', alpha=0.5, label=label)
        ax.annotate('', xy=(x_pos + 0.02, mean_tprs2[threshold + 1]), xycoords='data',
                     xytext=(x_pos + 0.02, mean_tprs[threshold]), textcoords='data', arrowprops={'arrowstyle': '|-|'})
        ax.annotate('NRI$_{events}$ = %0.2f' % (mean_tprs[threshold] - mean_tprs2[threshold]),
                     xy=(x_pos + x_offset, text_y_events), xycoords='data',
                     xytext=(x_pos + x_offset, text_y_events),
                     textcoords='offset points', fontsize=10)
        ax.annotate('', xy=(x_pos + 0.02, mean_fprs[threshold]), xycoords='data',
                     xytext=(x_pos + 0.02, mean_fprs2[threshold]), textcoords='data',
                     arrowprops=dict(arrowstyle='|-|', color='r'))
        ax.annotate('NRI$_{nonevents}$ = %0.2f' % (mean_fprs2[threshold] - mean_fprs[threshold]),
                     xy=(x_pos + x_offset2, text_y_nonevents), xycoords='data',
                     xytext=(x_pos + x_offset2, text_y_nonevents),
                     textcoords='offset points', fontsize=10)
        return x_pos, mean_tprs[threshold] - mean_tprs2[threshold], mean_fprs2[threshold] - mean_fprs[threshold], (mean_tprs[threshold] - mean_tprs2[threshold]) + (mean_fprs2[threshold] - mean_fprs[threshold])
        
    def generate_distinct_colors(list_size: int, avoid_red: bool = True, avoid_dark: bool = True) -> list:
        """
        Generate a list of distinct colors, optionally avoiding red and dark (near black) colors.
    
        Args:
            list_size (int): The number of distinct colors to generate.
            avoid_red (bool): Whether to avoid red colors. Defaults to True.
            avoid_dark (bool): Whether to avoid dark colors. Defaults to True.
    
        Returns:
            list: A list of colors in hexadecimal format.
        """
        colors = []
        # Define the HSV range to avoid red and dark colors
        hue_start = 0.1 if avoid_red else 0
        hue_end = 0.9 if avoid_red else 1
        saturation = 0.7
        value = 0.9 if avoid_dark else 0.5
        # Use HSV tuples to evenly space out colors and convert to RGB
        for i in np.linspace(hue_start, hue_end, list_size, endpoint=False):
            rgb_color = mcolors.hsv_to_rgb((i, saturation, value))  # Saturation and Value adjusted for color brightness
            hex_color = mcolors.rgb2hex(rgb_color)
            colors.append(hex_color)
        
        return colors
        
    colors = generate_distinct_colors(len(thresholds))
    nris = {}
    for c, i in enumerate(thresholds):
        thresh, nri_events, nri_nonevents, total_nri = nri_annotation(plt, int(i), thresholds[i], colors[c])
        nris[thresh] = [nri_events, nri_nonevents, total_nri]
        
    ax.set_xlim([0.0, 1.10])
    ax.set_ylim([0.0, 1.10])
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.set_xlabel('Calculated Risk', fontsize=18)
    ax.set_ylabel('Sensitivity (black), 1 - Specificity (red)', fontsize=18)
    ax.legend(loc="upper right", fontsize=11)
    ax.legend(loc=0, fontsize=11, bbox_to_anchor=(0, 0, 1.2, .9))
    ax.set_aspect('equal', adjustable='box')
    if save:
        fig.savefig('idi_curve.png', dpi=300, bbox_inches='tight')
    if show:
        print('IS positive', round(is_pos, 2), 'IS negative', round(is_neg, 2), 'IDI events', round(idi_event, 2))
        print('IP positive', round(ip_pos, 2), 'IP negative', round(ip_neg, 2), 'IDI nonevents', round(idi_nonevent, 2))
        print('IDI =', round(idi_event + idi_nonevent, 2))
        fig.show()
        
    idi_stats = {'IDI':idi_event + idi_nonevent, 'IDI Events':idi_event, 'IDI Nonevents':idi_nonevent, 'IS Positive':is_pos, 'IS Negative':is_neg, 'IP Positive':ip_pos, 'IP Negative':ip_neg}
    outputs = {'plot':fig,'IDI':idi_stats,'NRI':nris}
    return outputs #idi_event + idi_nonevent,idi_event, idi_nonevent, is_pos, is_neg, ip_pos, ip_neg, fig